<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# Figure 4: Future projections

_Authors: Oriana Chegwidden and Jeremy Freeman_

The methods below conduct the analyses to recreate Figure 4 included the
manuscript <<< insert doi >>>. Future risks for fire. We show median changes in
risk with respect to the historic.


In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import scipy as sp
from carbonplan_forests import load, setup, plot, fit, utils, prepare, collect
import xarray as xr
from carbonplan_forests.utils import get_store


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from carbonplan_forests import load, setup, plot, fit, utils, prepare, collect
from showit import image
import scipy as sp
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import regionmask as rm
from scipy.stats import binom
import altair as alt

from palettable.colorbrewer.sequential import YlOrRd_9

In [ ]:
coarsen = 4
store = "az"
tlim = ("1984", "2018")

In [ ]:
mask = load.mask(store=store, year=2001)

In [ ]:
nlcd = load.nlcd(store=store, year=2001, coarsen=coarsen, mask=mask)
nftd = load.nftd(store=store, area_threshold=1500, coarsen=coarsen, mask=mask)

In [ ]:
mask_coarsened = load.mask(store=store, year=2001, coarsen=coarsen)

In [ ]:
simulation = xr.open_zarr(
    get_store("carbonplan-scratch", "data/fire_future.zarr")
).assign_coords({"x": mask_coarsened.x, "y": mask_coarsened.y})

In [ ]:
mtbs = load.mtbs(store=store, coarsen=coarsen, tlim=tlim, mask=mask)
# give it the same x/y coords as nftd (and nlcd and climate)
# this re-assignment will help us with some masking lower down
mtbs = mtbs.assign_coords({"x": nftd.x, "y": nftd.y})

In [ ]:
alt.data_transformers.disable_max_rows()

insert pseudocode


In [ ]:
variables = {
    "ssp245": [
        "CanESM5",
        "FGOALS-g3",
        "HadGEM3-GC31-LL",
        "MIROC-ES2L",
        "MRI-ESM2-0",
    ],
    "ssp370": ["CanESM5", "MIROC-ES2L", "MRI-ESM2-0"],
    "ssp585": ["FGOALS-g3", "MIROC-ES2L", "MRI-ESM2-0"],
}

In [ ]:
# HOT TIP: coarsen is your friend! don't use resample! it does funky stuff with the first timechunk and you can't trust it boooo
# divide by 10 because you've grabbed 10 years but we still want it in annual averages (just at the decadal timestep)
decadal_averages = (
    simulation.sel(time=slice("2020", "2099")).coarsen(time=120).sum().load()
    / 10
)

In [ ]:
df = decadal_averages.where(mask_coarsened).mean(dim=["x", "y"]).to_dataframe()

In [ ]:
df["time"] = df.index
df_toplot = df.melt("time", var_name="gcm_scenario", value_name="probability")

In [ ]:
df_toplot["gcm"] = df_toplot.apply(
    lambda row: row.gcm_scenario.split("_")[0], axis=1
)
df_toplot["scenario"] = df_toplot.apply(
    lambda row: row.gcm_scenario.split("_")[1], axis=1
)

In [ ]:
import altair as alt

alt.themes.enable("carbonplan_light")

In [ ]:
historical_ds = xr.open_zarr(
    get_store("carbonplan-scratch", "data/fire.zarr")
).assign_coords({"x": mask_coarsened.x, "y": mask_coarsened.y})

In [ ]:
coarsened_historical = (
    historical_ds.sel(time=slice("1989", "2018")).coarsen(time=120).sum().load()
    / 10
)
historical_df = coarsened_historical.mean(dim=["x", "y"]).to_dataframe()

In [ ]:
decadal_averages = plot.fire.calc_decadal_averages(simulation)

In [ ]:
plot.fire.future_ts(
    decadal_averages.where(mask_coarsened), historical=historical_df
)